In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 4, Finished, Available, Finished)

In [3]:
df = spark.read.csv('Files/MtoMActual.csv', header=True)
display(df)

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5c90bc5b-53b8-4429-9464-511d2b67e6b4)

In [4]:
df.write.mode('overwrite').format('delta').saveAsTable('Mtomactual')

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 6, Finished, Available, Finished)

In [5]:
df = spark.sql("SELECT * FROM mylakehouse.MtoMActual")
display(df)

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 871ee59c-9974-42cc-8e7f-bd6884403748)

In [6]:
# dfreduced = df.select('Country', 'Actual') # not case sensitive
dfreduced = df.select(df.Country, df.Actual.alias('ActualSales')) # is case sensitive
# 当需要alias的时候使用下面的方式会更方便
# 'Actual'.alias是会报错
# display(dfreduced.filter("country = 'England'")) # not case sensitive
display(dfreduced.filter(dfreduced.Country != 'England')) # is case sensitive
# 这里要注意是 df.Country (这样写虽然结果对的但是不是很好）还是dfreduced.Country
# 还有就是需要用 == ,不能用 <>

# display(dfreduced.where("Country = 'England'"))
# filter 和 where 在这里是等价的，只是写法不同，这种写法可以<> 或者!=表示不等

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 96cdcdd4-b6ff-4c38-b596-e4f2e586ea53)

In [7]:
display(dfreduced.limit(2)) # 显示最上面的两个
display(dfreduced.tail(2)) # 显示最下面的两个

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f8ac8b7b-6650-412a-9a02-54bfe6f6bfbd)

SynapseWidget(Synapse.DataFrame, 19380137-c593-47f6-819f-404c4c8e4fc4)

In [8]:
%%sql
SELECT Country, Actual 
-- TOP(2) 是不工作的
As ActualSales
-- As `Actual Sales`
-- 不推荐header名字有空格
FROM mtomactual
WHERE Actual > 1000
LIMIT(2)

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 10, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 2 fields>

### Enriching data by adding new columns

In [9]:
display(df.withColumn("ActualDoubled", df.Actual * 2)\
          .withColumn("SecondLetter", df.Country.substr(2,1)) # startposition, length
        )


StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e5ad049c-583f-4e05-b100-02a12f85d1d7)

In [10]:
%%sql
SELECT *, 
    Actual * 2 AS ActualDoubled,
    SUBSTR(Country, 2, 1) AS SecondLetter
FROM mtomactual

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 12, Finished, Available, Finished)

<Spark SQL result set with 6 rows and 5 fields>

### Using Function

In [12]:
from pyspark.sql.functions import *
df2 = df.withColumn("ColDate", add_months(lit("2028-01-01"), df.Actual/1000))
display(df2)
df2.write.mode("overwrite").format("delta").saveAsTable("MtoMActualWithDates")

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 26964d1f-7eba-4f80-9b5a-f5a1b9b71b25)

In [14]:
df = spark.read.table('mtomactualwithdates')
display(df)

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2ebd5587-6ce9-458e-871e-abc933b20997)

### More Advanced filtering

In [31]:
display(df.where(df.Actual.between(6000, 11000)))
display(df.where(df.Country.contains("n")))
# 这个和 display(df.where(df.Country.like("%n%"))) 结果几乎一样
display(df.where(df.Country.like("%n_"))) # 但这个结果不一样，先出现一个 n，后面必须有且只有一个任意字符

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f2a83967-ca03-4859-a327-e62fc400c715)

SynapseWidget(Synapse.DataFrame, 88a748c8-844e-4aee-9009-04e2e434947f)

SynapseWidget(Synapse.DataFrame, 9e4a33d5-b971-4b91-a6a7-c077e1d025ec)

In [37]:
display(df.where("Country = 'England' or Country = 'France'")) # 这里注意 England 和France 字段要用单引号否则有syntax错误
display(df.where((df.Country == 'England') | (df.Country == 'France'))) # | or, & and, ~ not
display(df.where(df.Country.isin('England', 'France')))

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 40, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 08069d96-4843-44ec-8636-2b873a18d68a)

SynapseWidget(Synapse.DataFrame, 4710c188-9009-44b7-aff4-c8ed8c239ab0)

SynapseWidget(Synapse.DataFrame, 229f5d5b-4d80-4383-ad45-97dc6ab2380d)

In [38]:
%%sql
SELECT * 
FROM mtomactualwithdates
-- WHERE Actual BETWEEN 6000 and 11000
-- WHERE Country LIKE '%n%'
WHERE Country LIKE '%n_';

SELECT * 
FROM mtomactualwithdates
-- WHERE Country = "England" or Country = "France"
WHERE Country IN ("England", "France")

StatementMeta(, 9ddb003d-3624-4407-a7d8-742e6dd91652, 42, Finished, Available, Finished)

<Spark SQL result set with 3 rows and 4 fields>

<Spark SQL result set with 4 rows and 4 fields>